<a href="https://colab.research.google.com/github/toanpt74/COLAB_RD/blob/main/Unet3Plus_For_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import keras.metrics
from keras.layers import Conv2DTranspose, Activation, BatchNormalization, Conv2D, concatenate,Input,MaxPool2D, UpSampling2D,MaxPooling2D
from keras.models import Model
from keras import activations
import tensorflow as tf
import keras.backend as K
import os
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.callbacks import ModelCheckpoint
import cv2
import numpy as np

def LoadDataToTrain(input_shape=(256,256,1), path='', test_ratio=0.2, Use_Region = False):
    images_path = path
    X_path = [os.path.join(root, file) for root, _, files in
              os.walk(os.path.join(images_path, 'train')) for file in files if
              (file.endswith('.jpg') or file.endswith('.bmp') or file.endswith('.png'))]
    Y_path = [os.path.join(root, file) for root, _, files in
              os.walk(os.path.join(images_path, 'label')) for file in files if
              (file.endswith('.jpg') or file.endswith('.bmp') or file.endswith('.png'))]

    train_img_paths, test_img_paths, train_label_paths, test_label_paths = train_test_split(X_path, Y_path,test_size=0.2)

    if Use_Region == True:
        X_train = loadImageCrop(train_img_paths, input_shape=input_shape, islabel=0)
        Y_train = loadImageCrop(train_label_paths, input_shape=input_shape, islabel=1)
        X_test = loadImageCrop(test_img_paths, input_shape=input_shape, islabel=0)
        Y_test = loadImageCrop(test_label_paths, input_shape=input_shape, islabel=1)
    else:
        X_train = loadImages(train_img_paths, input_shape=input_shape, islabel=0)
        Y_train = loadImages(train_label_paths, input_shape=input_shape, islabel=1)
        X_test = loadImages(test_img_paths, input_shape=input_shape, islabel=0)
        Y_test = loadImages(test_label_paths, input_shape=input_shape, islabel=1)
    Y_train = np.where(Y_train < 1, 0.0, 1.0)
    Y_test = np.where(Y_test < 1, 0.0, 1.0)

    X_test = X_test.reshape((X_test.shape[0], input_shape[0], input_shape[1], 1))
    Y_test = Y_test.reshape((Y_test.shape[0], input_shape[0], input_shape[1], 1))
    X_train = X_train.reshape((X_train.shape[0], input_shape[0], input_shape[1], 1))
    Y_train = Y_train.reshape((Y_train.shape[0], input_shape[0], input_shape[1], 1))
    return (X_train, Y_train), (X_test, Y_test)
def LoadDataFromImage(input_shape=(256,256,1), path='', test_ratio=0.2):
    images_path = path

    X_path = [os.path.join(root, file) for root, _, files in
              os.walk(os.path.join(images_path, 'train')) for file in files if
              (file.endswith('.jpg') or file.endswith('.bmp') or file.endswith('.png'))]
    Y_path = [os.path.join(root, file) for root, _, files in
              os.walk(os.path.join(images_path, 'label')) for file in files if
              (file.endswith('.jpg') or file.endswith('.bmp') or file.endswith('.png'))]
    #Xu dung crop vung anh
    #X = loadImageCrop(X_path, train_region=train_region, input_shape=input_shape, islabel=0)
    #Y = loadImageCrop(Y_path, train_region=train_region, input_shape=input_shape, islabel=1)
    X = loadImages(X_path, input_shape=input_shape, islabel=0)
    Y = loadImages(Y_path, input_shape=input_shape, islabel=1)
    Y = np.where(Y < 0.5, 0.0, 1.0)
    return (X,Y)  #(X_train, Y_train), (X_test, Y_test)
def loadImages(images_path, input_shape, islabel=0):
    data = []
    print("**************")
    for file in images_path:
        print("Load file:" + file)
        im = cv2.imread(file)
        d = len(im.shape)
        if d == 3:
            im = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
        # Crop Image
        im = cv2.resize(im, (input_shape[1], input_shape[0]))
        im = np.array(im)
        im = im.astype('float32')
        temp = im / 255.0
        if islabel ==1 :
            print(temp.shape)
            print(np.sum(temp))
        data.append(temp)
    X = np.array(data)
    return X
def loadImageCrop(images_path, train_region, input_shape, islabel=0):
    data = []

    for file in images_path:
        print("Load file:" + file)
        im = cv2.imread(file)
        d = len(im.shape)
        if d == 3:
            im = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
        # Crop Image
        crop_img = im[train_region[1]:train_region[1] + train_region[3],
                   train_region[0]:train_region[0] + train_region[2]]

        crop_img = cv2.resize(crop_img, (input_shape[1], input_shape[0]))

        crop_img = np.array(crop_img)
        crop_img = crop_img.astype('float32')
        temp = crop_img / 255.0
        if islabel ==1 and np.sum(temp) <500:
            temp[:,:]=0.0

        data.append(temp)
    X = np.array(data)
    return X
def dice_coef(y_true, y_pred):
    smooth = 1e-6
    intersection = K.sum(y_true * y_pred)
    dice_loss= (2. * intersection + smooth) / (K.sum(y_true) + K.sum(y_pred) + smooth)

    #dice_loss = (2. * K.sum(y_true * y_pred) + 1.) / (K.sum(y_true) + K.sum(y_pred) + 1.)
    return  1- dice_loss
def IoULoss(targets, inputs):
    # flatten label and prediction tensors
    # inputs = K.flatten(inputs)
    # targets = K.flatten(targets)
    smooth = 1e-6
    ##intersection = K.sum(K.dot(targets, inputs))
    intersection = K.sum(targets*inputs)
    total = K.sum(targets) + K.sum(inputs)
    union = total - intersection
    IoU = (intersection + smooth) / (union + smooth)
    return 1-IoU

def conv_block(x, filters=16, kernel_size=(3, 3), padding='same', strides=(1, 1), dilation_rate=(1, 1), n=2,name='Conv', is_bn=True, is_relu=True):
    for i in range(1, n + 1):
        y = Conv2D(filters=filters, kernel_size=kernel_size, kernel_initializer='he_normal', padding=padding, strides=strides,
        dilation_rate=dilation_rate, name='{0}_{1}'.format(name, i))(x)
        if is_bn:
            y = BatchNormalization(name='BN_' + name)(y)
        if is_relu:
            y = Activation('relu', name='AC_' + name)(y)
    return y

def conv_block_small(x, filters=16, kernel_size=(3, 3), padding='same', strides=(1, 1), dilation_rate=(2, 2),name='Conv'):
    y = Conv2D(filters=filters, kernel_size=kernel_size, kernel_initializer='he_normal', padding=padding,
    strides=strides, dilation_rate=dilation_rate, name=name)(x)
    y = BatchNormalization(name='BN_' + name)(y)
    y = Activation('relu', name='AC_' + name)(y)
    return y
def wide_block(x, filters=16, name='Conv'):
    en1 = conv_block_small(x, filters=2 ** filters, kernel_size=(3, 3), name=name + 'en1', dilation_rate=(1, 1))  # 320*320*64
    en2 = conv_block_small(x, filters=2 ** (filters + 1), kernel_size=(3, 3), name=name + 'en2', dilation_rate=(2, 2))
    en3 = conv_block_small(x, filters=2 ** (filters + 2), kernel_size=(3, 3), name=name + 'en3', dilation_rate=(3, 3))
    e = concatenate(inputs=[en1, en2, en3], axis=-1)
    e = conv_block_small(x=e, filters=2 ** (filters + 2), kernel_size=(1, 1), name=name + 'e', dilation_rate=(1, 1))
    return e

filters = [2, 3, 4, 5, 6]

def Assy_Model(INPUT_SHAPE, OUTPUT_CHANNELS):
    input_layer = Input(shape=INPUT_SHAPE, name="input_layer")
    """ Encoder"""
    # block 1
    e1 = wide_block(x=input_layer, filters=filters[0], name='Enc_{0}_1'.format(filters[0]))
    # block 2
    e2 = MaxPooling2D(pool_size=(2, 2))(e1)  # 160*160*64
    e2 = wide_block(e2, filters[1], name='Enc_{0}_1'.format(filters[1]))
    # block 3
    e3 = MaxPooling2D(pool_size=(2, 2))(e2)  # 80*80*128
    e3 = wide_block(e3, filters[2], name='Enc_{0}_1'.format(filters[2]))
    # block 4
    e4 = MaxPooling2D(pool_size=(2, 2))(e3)  # 40*40*256
    e4 = wide_block(e4, filters[3], name='Enc_{0}_1'.format(filters[3]))
    # block 5
    e5 = MaxPooling2D(pool_size=(2, 2))(e4)  # 20*20*512
    e5 = wide_block(e5, filters[4], name='Enc_{0}_1'.format(filters[4]))
    """ Decoder """
    cat_channels = 32
    upsample_channels = 64
    """ d4 """
    e1_d4 = MaxPooling2D(pool_size=(8, 8))(e1)
    e1_d4 = conv_block(e1_d4, cat_channels, n=1, name='Dec_e1_d4_1')
    e2_d4 = MaxPooling2D(pool_size=(4, 4))(e2)
    e2_d4 = conv_block(e2_d4, cat_channels, n=1, name='Dec_e2_d4_1')
    e3_d4 = MaxPooling2D(pool_size=(2, 2))(e3)
    e3_d4 = conv_block(e3_d4, cat_channels, n=1, name='Dec_e3_d4_1')
    e4_d4 = conv_block(e4, cat_channels, n=1)
    e5_d4 = UpSampling2D(size=(2, 2), interpolation='bilinear')(e5)
    e5_d4 = conv_block(e5_d4, cat_channels, n=1, name='Dec_e5_d4_1')
    d4 = concatenate([e1_d4, e2_d4, e3_d4, e4_d4, e5_d4])
    d4 = conv_block(d4, upsample_channels, n=1, name='Dec_d4_1')
    """ d3 """
    e1_d3 = MaxPooling2D(pool_size=(4, 4))(e1)
    e1_d3 = conv_block(e1_d3, cat_channels, n=1, name='Dec_e1_d3_1')
    e2_d3 = MaxPooling2D(pool_size=(2, 2))(e2)
    e2_d3 = conv_block(e2_d3, cat_channels, n=1, name='Dec_e2_d3_1')
    e3_d3 = conv_block(e3, cat_channels, n=1, name='Dec_e3_d3_1')
    e4_d3 = UpSampling2D(size=(2, 2), interpolation='bilinear')(d4)
    e4_d3 = conv_block(e4_d3, cat_channels, n=1, name='Dec_e4_d3_1')
    e5_d3 = UpSampling2D(size=(4, 4), interpolation='bilinear')(e5)
    e5_d3 = conv_block(e5_d3, cat_channels, n=1, name='Dec_e5_d3_1')
    d3 = concatenate([e1_d3, e2_d3, e3_d3, e4_d3, e5_d3])
    d3 = conv_block(d3, upsample_channels, n=1, name='Dec_d3_1')
    """ d2 """
    e1_d2 = MaxPooling2D(pool_size=(2, 2))(e1)
    e1_d2 = conv_block(e1_d2, cat_channels, n=1, name='Dec_e1_d2_1')
    e2_d2 = conv_block(e2, cat_channels, n=1, name='Dec_e2_d2_1')
    d3_d2 = UpSampling2D(size=(2, 2), interpolation='bilinear')(d3)
    d3_d2 = conv_block(d3_d2, cat_channels, n=1, name='Dec_d3_d2_1')
    d4_d2 = UpSampling2D(size=(4, 4), interpolation='bilinear')(d4)
    d4_d2 = conv_block(d4_d2, cat_channels, n=1, name='Dec_d4_d2_1')
    e5_d2 = UpSampling2D(size=(8, 8), interpolation='bilinear')(e5)
    e5_d2 = conv_block(e5_d2, cat_channels, n=1, name='Dec_e5_d2_1')
    d2 = concatenate([e1_d2, e2_d2, d3_d2, d4_d2, e5_d2])
    d2 = conv_block(d2, upsample_channels, n=1, name='Dec_d2_1')
    """ d1 """
    e1_d1 = conv_block(e1, cat_channels, n=1, name='Dec_e1_d1_1')
    d2_d1 = UpSampling2D(size=(2, 2), interpolation='bilinear')(d2)
    d2_d1 = conv_block(d2_d1, cat_channels, n=1, name='Dec_d2_d1_1')
    d3_d1 = UpSampling2D(size=(4, 4), interpolation='bilinear')(d3)
    d3_d1 = conv_block(d3_d1, cat_channels, n=1, name='Dec_d3_d1_1')
    d4_d1 = UpSampling2D(size=(8, 8), interpolation='bilinear')(d4)
    d4_d1 = conv_block(d4_d1, cat_channels, n=1, name='Dec_d4_d1_1')
    e5_d1 = UpSampling2D(size=(16, 16), interpolation='bilinear')(e5)
    e5_d1 = conv_block(e5_d1, cat_channels, n=1, name='Dec_e5_d1_1')
    d1 = concatenate([e1_d1, d2_d1, d3_d1, d4_d1, e5_d1])
    d1 = conv_block(d1, upsample_channels, n=1, name='Dec_d1_1')
    # last layer does not have batchnorm and relu
    d = conv_block(d1, OUTPUT_CHANNELS, n=1, is_bn=False, is_relu=False, name='Dec_d_1')
    if OUTPUT_CHANNELS == 1:
        output = activations.sigmoid(d)
    else:
        output = activations.softmax(d)
    return Model(inputs=input_layer, outputs=output, name='UNet_3Plus')

epochs=201
BATCH_SIZE=1
OUTPUT_CHANNELS=1
INPUT_SHAPE = [512, 512, 1]
OPTIMIZER = tf.keras.optimizers.Adam()
path= r'E:\data'
model_save_path = r'E:\ToanPT\1.Code_train_Unet\models\unet'
model = Assy_Model(INPUT_SHAPE, OUTPUT_CHANNELS)
model.compile(optimizer=OPTIMIZER, loss = 'binary_crossentropy', metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()])
#model.compile(optimizer=OPTIMIZER, loss=IoULoss,metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()])
model.summary()

callbacks = [tf.keras.callbacks.TensorBoard(log_dir='./logs'),
                   ModelCheckpoint(
                       os.path.join(model_save_path,
                     'model-assy-{epoch:03d}--{loss:.6f}-{accuracy:.6f}--{val_loss:.6f}-{val_accuracy:.6f}.h5'),
                       monitor='val_accuracy', save_best_only=False,
                       save_weights_only=False, period=5, mode='auto', verbose=0),
                  ]
(x_train, y_train), (x_test, y_test) = LoadDataToTrain(input_shape=INPUT_SHAPE,path=path, Use_Region=False)
model.fit(x_train, y_train, validation_data=(x_test, y_test), verbose=1, epochs=epochs, batch_size=BATCH_SIZE, shuffle=True, callbacks=callbacks)